# Run QARTOD Test on Locally Saved Data

In this notebook we use the paths to locally saved data in the external data folder to extract QARTOD test parameters, download parameter thresholds on the OOI GitHub, run the QARTOD climatology and gross range tests on the imported data, and save the test results to the interim data folder.

More info about QARTOD tests and the ioos_qc module can be found from the [Integrated Ocean Observing System website](https://ioos.noaa.gov/project/qartod/) and [Python module documentation](https://ioos.github.io/ioos_qc/), respectively.

### Import modules for local QC test

In [1]:
# Import libraries
import os
import warnings
warnings.filterwarnings("ignore")
import glob

# Import qartod_testing project functions
from qartod_testing.local_qc_test import save_local_qc_tests

### Find locally saved data downloaded from OOINet

In [2]:
# Set reference designator, data stream, and method 
refdes = "GA03FLMB-RIS01-04-PHSENF000"        
method = "recovered_inst"
stream = "phsen_abcdef_instrument"

In [3]:
# Build path to folder where data was saved
folder_path = os.path.join(os.path.abspath('../data/external'), method,
                           stream, refdes)

# Retrieve a list of netCDF files in this directory
file_paths = glob.glob(folder_path+'/*.nc')
file_paths.sort()

### Run QARTOD tests locally
Next, we run the QC tests locally to get our own ("local") results that can be compared with the production ("expected") results from the datasets downloaded from OOINet. This is done using the climatology or gross range tests from the ```ioos_qc``` QARTOD package in conjunction with the ```qartod_test_values``` tables on the oceanobservatories/qc-lookup repository on GitHub. All of the requests for the lookup tables are done within our functions to configure and run the local tests. ```save_local_qc_tests``` will identify which parameters in the dataset have QARTOD applied to them from variable names that contain ```qartod_executed``` to run the local tests on just these parameters.

#### Gross Range Test

In [4]:
# Run local QARTOD gross range tests
save_local_qc_tests(file_paths, refdes, stream, method, "gross_range")

Running local QARTOD gross_range test for deployment 01.
Running local QARTOD gross_range test for deployment 02.
Running local QARTOD gross_range test for deployment 03.


#### Climatology Test

In [5]:
# Run local QARTOD climatology tests
save_local_qc_tests(file_paths, refdes, stream, method, "climatology")

Running local QARTOD climatology test for deployment 01.
Running local QARTOD climatology test for deployment 02.
Running local QARTOD climatology test for deployment 03.
